In [1]:
from Bio import Entrez
import re

Entrez.email = "emirhan.acikgoez@hotmail.com"  # Always tell NCBI who you are

# Search PubMed for articles with "GC content of" or "G + C content of" in the title or abstract
abstracts_with_gc = []
for i in range(0, 200, 20):
    handle = Entrez.esearch(db="pubmed", term='("GC content of" OR "G + C content of") AND bacterial[Title/Abstract]', retstart=i, retmax=20)
    record = Entrez.read(handle)
    ids = record["IdList"]
    # Process the IDs

    # Get the list of article IDs
    article_ids = record["IdList"]

    # Fetch the abstracts for the articles
    handle = Entrez.efetch(db="pubmed", id=article_ids, rettype="abstract")
    records = Entrez.read(handle)
    
    # Parse the abstracts
    for record in records["PubmedArticle"]:
        try:
            temp_record = record["MedlineCitation"]["Article"]
            abstract = temp_record["Abstract"]["AbstractText"][0]
            if re.search(r"\bGC\scontent\sof\b", abstract, re.IGNORECASE) and len(re.findall(r"([\d\.]+\%)", abstract)) == 1:
                bacterial_match = re.search(r"<i>(.*?)</i>", abstract)
                doi = record['PubmedData']['ArticleIdList'][0] if 'ArticleIdList' in record['PubmedData'] else 'N/A'
                if bacterial_match:
                    bacterial_name = bacterial_match.group(1)
                    # Prompt the user to input whether the GC content value is correct or not
                    print(f"DOI: {doi}")
                    correct = input(f"Is the GC content value 'GC content of' correct for {bacterial_name}? (y/n) ")
                    while correct.lower() not in ['y', 'n']:
                        correct = input("Invalid input. Please enter 'y' or 'n': ")
                    if correct.lower() == 'y':
                        abstracts_with_gc.append((bacterial_name, 'GC content of'))
        except KeyError:
            print("No abstract available")

# Print the bacterial names and their corresponding GC content phrases
for bacterial_name, gc_content in abstracts_with_gc:
    print(f"{bacterial_name} has the phrase 'GC content of'")

KeyboardInterrupt: 

# Offene Fragen


Ansatz um Daten aus PubMed zu extrahieren und mit Doccano zu annotieren:

Datenextraktion aus PubMed:

Verwenden Sie die PubMed-API oder eine geeignete Python-Bibliothek wie Biopython oder Entrez, um auf die gewünschten Publikationen zuzugreifen.
Definieren Sie die Suchkriterien, um Artikel mit dem gewünschten Inhalt zu finden, z.B. "GC content of" im Abstract und "bacterial" im Titel.
Holen Sie die relevanten Artikel basierend auf den definierten Kriterien.
Vorbereitung der Daten für Doccano:

Extrahieren Sie die Titel und Abstracts der ausgewählten Artikel.
Speichern Sie die extrahierten Texte in einem geeigneten Format (z.B. JSONL), das von Doccano akzeptiert wird. Stellen Sie sicher, dass jede Texteinheit (z. B. ein Abstract) einen eindeutigen Bezeichner hat.
Einrichtung von Doccano:

Installieren und konfigurieren Sie Doccano auf Ihrem gewählten Server oder lokalen Rechner. Weitere Informationen zur Installation finden Sie in der offiziellen Doccano-Dokumentation.
Erstellen eines Projekts und Annotationsschemas:

Öffnen Sie Doccano in Ihrem Webbrowser und erstellen Sie ein neues Projekt.
Definieren Sie ein passendes Annotationsschema, das Ihre Anforderungen erfüllt. Fügen Sie beispielsweise Annotationstypen für "GC content of" und "bacterial" hinzu.
Hochladen der extrahierten Daten:

Laden Sie das zuvor vorbereitete Datenformat (z.B. JSONL) in Doccano hoch, indem Sie die entsprechende Option im Projektmenü auswählen.
Überprüfen Sie die hochgeladenen Daten, um sicherzustellen, dass die Texteinheiten korrekt angezeigt werden.
Annotation der Daten:

Öffnen Sie die einzelnen Texteinheiten (z.B. Abstracts) in Doccano und beginnen Sie mit der Annotation basierend auf Ihrem definierten Schema.
Markieren Sie die gewünschten Textstellen im Abstract, die den Kriterien "GC content of" und "bacterial" entsprechen.
Zusammenarbeit und Überprüfung (optional):

Wenn mehrere Personen an der Annotation beteiligt sind, können Sie die Zusammenarbeit in Doccano aktivieren, um die Annotationen zu überprüfen und zu überarbeiten.
Export der annotierten Daten:

Exportieren Sie die annotierten Daten aus Doccano, um sie in einem geeigneten Format zu speichern (z.B. JSONL) und sie für weitere Verarbeitungsschritte oder das Training von ML-Modellen zu verwenden.
Dieser Prozess sollte es Ihnen ermöglichen, die gewünschten Abstrakte aus PubMed zu extrahieren, sie in Doccano hochzuladen und die Annotationen gemäß Ihren spezifischen Anforderungen durchzuführen.

In [ ]:
# Code:
from Bio import Entrez
import json
import requests

# PubMed-Abfrageparameter
query = '"GC content of"[Abstract] AND bacterial[Title]'
email = 'your_email@example.com'  # Geben Sie Ihre E-Mail-Adresse ein

# PubMed-Daten abrufen
def fetch_pubmed_data(query, email):
    Entrez.email = email
    handle = Entrez.esearch(db='pubmed', retmax=10, term=query)
    record = Entrez.read(handle)
    id_list = record['IdList']
    handle = Entrez.efetch(db='pubmed', id=id_list, rettype='abstract', retmode='text')
    abstracts = handle.readlines()
    return abstracts

# Doccano-Datenformat erstellen
def create_doccano_data(abstracts):
    doccano_data = []
    for idx, abstract in enumerate(abstracts):
        data = {
            'text': abstract.strip(),
            'meta': {'source': f'PubMed Abstract {idx+1}'}
        }
        doccano_data.append(data)
    return doccano_data

# Doccano-Daten exportieren
def export_doccano_data(doccano_data, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for data in doccano_data:
            json.dump(data, file, ensure_ascii=False)
            file.write('\n')

# PubMed-Daten abrufen
abstracts = fetch_pubmed_data(query, email)

# Doccano-Daten erstellen
doccano_data = create_doccano_data(abstracts)

# Doccano-Daten exportieren (im JSONL-Format)
export_doccano_data(doccano_data, 'doccano_data.jsonl')
